# Simulation with traffic reading. 

This demo is for the case reading a recorded traffic flow and passing them over a bridge. 

The bridge: 

- Has four 3.5m-width lanes. 
- Has a length of 20m.
- Has a width of 16m.
- Has one load effect being considered. 

The traffic flow:

- It is a history traffic. 

---

Same processes for the bridge definition as the previous demo.

In [4]:
import pybtls as pb
from pathlib import Path


# set the load effect by using a 2D influence surface
lanes_position = [(0.5, 4.0), (4.0, 7.5), (8.5, 12.0), (12.0, 15.5)]
IS_matrix = [
    [0.0, 0.0, 4.0, 8.0, 12.0, 16.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [10.0, 0.0, 2.5, 5.0, 2.5, 0.0],
    [20.0, 0.0, 0.0, 0.0, 0.0, 0.0],
]
load_effect = pb.InfluenceSurface()
load_effect.set_IS(IS_matrix, lanes_position)

# set the bridge
bridge = pb.Bridge(length=20.0, no_lane=4)
bridge.add_load_effect(inf_line_surf=load_effect, threshold=0.0)


Load a history traffic flow.

In [5]:
traffic_loader = pb.TrafficLoader(no_lane=4)
traffic_loader.add_traffic(
    traffic=Path(".") / "test_traffic_file.txt",
    traffic_format=4,
    # use_average_speed=False,  # optional
    # use_const_speed=False,  # optional
    # const_speed_value=36.0,  # optional, valid if use_const_speed=True
)


Set output. Here we output all possible results. 

In [6]:
output_config = pb.OutputConfig()
output_config.set_event_output(write_time_history=True, write_each_event=True)
output_config.set_vehicle_file_output(
    write_vehicle_file=True,
    vehicle_file_format=4,
    vehicle_file_name="proof_traffic_file.txt",
)  # you will see the proof_traffic_file.txt is the same as the test_traffic_file.txt
output_config.set_BM_output(
    write_vehicle=True, write_summary=True, write_mixed=True
)
output_config.set_POT_output(
    write_vehicle=True, write_summary=True, write_counter=True
)
output_config.set_stats_output(
    write_flow_stats=True, write_overall=True, write_intervals=True
)
output_config.set_fatigue_output(
    write_fatigue_event=True, write_rainflow_output=True
)


Set simulation and get the outputs. 

In [7]:
sim_task = pb.Simulation(Path(".") / "temp")
sim_task.add_sim(
    bridge=bridge,
    traffic=traffic_loader,
    output_config=output_config,
    time_step=0.1,
    min_gvw=35,
    # active_lane=[1,2,3,4],  # optional, if not set, all lanes will be active.
    # track_progress=False,  # optional, if True, the progress print will show up.
    tag="Case3",
)

# run simulation
sim_task.run(no_core=1)

# get the result
sim_output = sim_task.get_output()

# See what is included
print(sim_output["Case3"].get_summary())



Bridge 20 m: Flushing AllEvents buffer: 1415 events at 1/1/0 23:57:48	
Bridge 20 m: Flushing Fatigue buffer: 1415 events at 1/1/0 23:57:48	
Flushing buffer of 9427 vehicles at 2/1/0 0:0:17.395
['time_history', 'all_events', 'traffic', 'BM_by_no_trucks', 'BM_by_mixed', 'BM_summary', 'POT_vehicle', 'POT_summary', 'POT_counter', 'traffic_statistics', 'E_cumulative_statistics', 'E_interval_statistics', 'fatigue_events', 'fatigue_rainflow']
